In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [2]:
current_dir = Path().resolve()

data_dir = current_dir / 'data'
os.chdir(data_dir)


In [3]:
os.listdir()

['Cities.csv',
 'Conferences.csv',
 'MConferenceTourneyGames.csv',
 'MGameCities.csv',
 'MMasseyOrdinals.csv',
 'MNCAATourneyCompactResults.csv',
 'MNCAATourneyDetailedResults.csv',
 'MNCAATourneySeedRoundSlots.csv',
 'MNCAATourneySeeds.csv',
 'MNCAATourneySlots.csv',
 'MRegularSeasonCompactResults.csv',
 'MRegularSeasonDetailedResults.csv',
 'MSeasons.csv',
 'MSecondaryTourneyCompactResults.csv',
 'MSecondaryTourneyTeams.csv',
 'MTeamCoaches.csv',
 'MTeamConferences.csv',
 'MTeams.csv',
 'MTeamSpellings.csv',
 'SampleSubmissionStage1.csv',
 'SeedBenchmarkStage1.csv',
 'WConferenceTourneyGames.csv',
 'WGameCities.csv',
 'WNCAATourneyCompactResults.csv',
 'WNCAATourneyDetailedResults.csv',
 'WNCAATourneySeeds.csv',
 'WNCAATourneySlots.csv',
 'WRegularSeasonCompactResults.csv',
 'WRegularSeasonDetailedResults.csv',
 'WSeasons.csv',
 'WSecondaryTourneyCompactResults.csv',
 'WSecondaryTourneyTeams.csv',
 'WTeamConferences.csv',
 'WTeams.csv',
 'WTeamSpellings.csv']

# EDA

In [4]:
files = os.listdir(data_dir)
dataframes = {}

for file in files:
    if file.endswith('.csv'):
        file_path = os.path.join(data_dir, file)
        df_name = os.path.splitext(file)[0]
        try:
            dataframes[df_name] = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            # Try a different encoding if utf-8 fails
            dataframes[df_name] = pd.read_csv(file_path, encoding='latin1')
            
        print(f'Loaded {file} into DataFrame {df_name}')


Loaded Cities.csv into DataFrame Cities
Loaded Conferences.csv into DataFrame Conferences
Loaded MConferenceTourneyGames.csv into DataFrame MConferenceTourneyGames
Loaded MGameCities.csv into DataFrame MGameCities
Loaded MMasseyOrdinals.csv into DataFrame MMasseyOrdinals
Loaded MNCAATourneyCompactResults.csv into DataFrame MNCAATourneyCompactResults
Loaded MNCAATourneyDetailedResults.csv into DataFrame MNCAATourneyDetailedResults
Loaded MNCAATourneySeedRoundSlots.csv into DataFrame MNCAATourneySeedRoundSlots
Loaded MNCAATourneySeeds.csv into DataFrame MNCAATourneySeeds
Loaded MNCAATourneySlots.csv into DataFrame MNCAATourneySlots
Loaded MRegularSeasonCompactResults.csv into DataFrame MRegularSeasonCompactResults
Loaded MRegularSeasonDetailedResults.csv into DataFrame MRegularSeasonDetailedResults
Loaded MSeasons.csv into DataFrame MSeasons
Loaded MSecondaryTourneyCompactResults.csv into DataFrame MSecondaryTourneyCompactResults
Loaded MSecondaryTourneyTeams.csv into DataFrame MSecondar

In [5]:
print('DataFrames available: ')
for df_name in dataframes.keys():
    print(df_name)

DataFrames available: 
Cities
Conferences
MConferenceTourneyGames
MGameCities
MMasseyOrdinals
MNCAATourneyCompactResults
MNCAATourneyDetailedResults
MNCAATourneySeedRoundSlots
MNCAATourneySeeds
MNCAATourneySlots
MRegularSeasonCompactResults
MRegularSeasonDetailedResults
MSeasons
MSecondaryTourneyCompactResults
MSecondaryTourneyTeams
MTeamCoaches
MTeamConferences
MTeams
MTeamSpellings
SampleSubmissionStage1
SeedBenchmarkStage1
WConferenceTourneyGames
WGameCities
WNCAATourneyCompactResults
WNCAATourneyDetailedResults
WNCAATourneySeeds
WNCAATourneySlots
WRegularSeasonCompactResults
WRegularSeasonDetailedResults
WSeasons
WSecondaryTourneyCompactResults
WSecondaryTourneyTeams
WTeamConferences
WTeams
WTeamSpellings
